In [1]:
%load_ext pycodestyle_magic

In [2]:
%flake8_on

In [3]:
from math import sqrt
from enum import Enum

In [4]:
testdata = """Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...""".splitlines()

with open('input', 'r') as inp:
    inputdata = [line.strip() for line in inp.readlines()]

monster_pattern = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """.splitlines()

112:23: W291 trailing whitespace


In [5]:
def parse(lines):
    blank_lines = [idx for idx, line in enumerate(lines)
                   if line == '']
    boundaries = zip(
        [0] + [idx + 1 for idx in blank_lines],
        blank_lines + [len(lines)]
    )

    tiles = dict()
    for low, high in boundaries:
        tile = Tile(lines[low:high])
        tiles[tile.id] = tile

    return tiles

In [6]:
class Directions(Enum):
    def __repr__(self):
        return self.name

    def flipped(self):
        return Directions((Directions.WEST.value - self.value) % 4)

    def translate(self, flipped, rotation):
        if not flipped:
            return Directions((self.value + rotation) % 4)
        else:
            return Directions((self.flipped().value + rotation) % 4)

    def rotation_to(self, flipped, target_direction):
        if not flipped:
            return (target_direction.value - self.value) % 4
        else:
            return (target_direction.value - self.flipped().value) % 4

    NORTH = 0
    EAST = 1
    SOUTH = 2
    WEST = 3

In [7]:
class Tile(object):
    def __init__(self, lines):
        self.id = int(lines[0][5:-1])
        self.lines = lines[1:]
        self.size = len(self.lines)
        self.values = []
        self.values_flipped = []
        self.is_flipped = False
        self.rotated = 0
        self.set_values()
        self.fits_with = []
        self.fits_with_flipped = []

    def __repr__(self):
        return (f'id: {str(self.id)}, values: {str(self.values)}' +
                f', values_flipped: {str(self.values_flipped)}' +
                f', fits_with: {str(self.fits_with)}' +
                f', fits_with_flipped: {str(self.fits_with_flipped)}'
                )

    def set_values(self):
        self.values = [0] * 4
        self.values_flipped = [0] * 4
        for direction in Directions:
            self.values[direction.value] = self.value(direction)
            self.values_flipped[direction.value] = \
                self.value_flipped(direction)

    def value(self, direction):
        return sum([1 << b for b, c
                    in enumerate(self.get_edge(direction)[::-1])
                    if c == '#'])

    def value_flipped(self, direction):
        return sum([1 << b for b, c
                    in enumerate(self.get_edge(direction))
                    if c == '#'])

    def get_edge(self, direction):
        if direction == Directions.NORTH:
            return self.lines[0]
        elif direction == Directions.EAST:
            return [line[-1] for line in self.lines]
        elif direction == Directions.SOUTH:
            return self.lines[-1][::-1]
        elif direction == Directions.WEST:
            return [line[0] for line in self.lines[::-1]]

    def get_as_placed(self, flipped, rotation):
        rotation = rotation % 4
        output = self.lines[:]

        output_flipped = output[:]
        if flipped:
            for i in range(len(output)):
                output_flipped[i] = ''.join(
                    [line[i] for line in output]
                    )

        output_rotated = output_flipped[:]
        if rotation == 1:
            for i in range(len(output)):
                output_rotated[i] = ''.join(
                    [line[i] for line in output_flipped[::-1]]
                    )
        elif rotation == 2:
            for i in range(len(output)):
                output_rotated[i] = output_flipped[-(1 + i)][::-1]
        elif rotation == 3:
            for i in range(len(output)):
                output_rotated[i] = ''.join(
                    [line[-(1 + i)] for line in output_flipped]
                    )

        return output_rotated

    def fit_with(self, other):
        for i in Directions:
            for j in Directions:
                if (self.values[i.value]
                        == other.values_flipped[j.value]):
                    self.fits_with.append((i, other.id, j))
                    other.fits_with.append((j, self.id, i))
                if (self.values_flipped[i.value]
                        == other.values_flipped[j.value]):
                    self.fits_with_flipped.append((i, other.id, j))
                    other.fits_with_flipped.append((j, self.id, i))

    def translate_fits_with(fits_with, flipped, rotation):
        return [(our_edge.translate(flipped, rotation),
                 other_id,
                 other_edge)
                for our_edge, other_id, other_edge in fits_with]

    def get_fits_with_transated(self, flipped, rotation):
        if not flipped:
            return (Tile.translate_fits_with(self.fits_with,
                                             flipped,
                                             rotation),
                    Tile.translate_fits_with(self.fits_with_flipped,
                                             flipped,
                                             rotation))
        else:
            return (Tile.translate_fits_with(self.fits_with_flipped,
                                             flipped,
                                             rotation),
                    Tile.translate_fits_with(self.fits_with,
                                             flipped,
                                             rotation))

In [8]:
class Solution(object):
    def __init__(self, tiles):
        self.size = round(sqrt(len(tiles)))
        self.tiles = tiles
        self.placements = [[(0, 0, False)
                            for x in range(self.size)]
                           for y in range(self.size)]

    def fit_tiles(self, silent=False):
        corner_ids = []
        tile_list = list(self.tiles.values())
        for i in range(len(tile_list) - 1):
            for j in range(i + 1, len(tile_list)):
                tile_list[i].fit_with(tile_list[j])
        for tile in tile_list:
            if not silent:
                print(f'id: {tile.id}, fits_with: {tile.fits_with}' +
                      f', fits_with_flipped: {tile.fits_with_flipped}')
            if len(tile.fits_with) + len(tile.fits_with_flipped) == 2:
                corner_ids.append(tile.id)
        return corner_ids

    def look_direction(self, x, y):
        if x > 0 and self.placements[y][x - 1][0]:
            return (Directions.WEST, x - 1, y)
        elif y > 0 and self.placements[y - 1][x][0]:
            return (Directions.NORTH, x, y - 1)
        elif x < self.size and self.placements[y][x + 1][0]:
            return (Directions.EAST, x + 1, y)
        elif y < self.size and self.placements[y + 1][x][0]:
            return (Directions.SOUTH, x, y + 1)

    def place_by_tile(look_direction, fits_with, flipped):
        for previous_edge, our_id, our_edge in fits_with:
            if previous_edge == look_direction.translate(False, 2):
                our_rotation = our_edge.rotation_to(flipped, look_direction)
                return our_id, flipped, our_rotation
        return False

    def place_by_previous(self, x, y):
        look_direction, look_x, look_y = self.look_direction(x, y)
        previous_id, previous_flipped, previous_rotation \
            = self.placements[look_y][look_x]
        previous_tile = self.tiles[previous_id]
        # print(f'previous_tile: {previous_tile}')
        previous_fits_with, previous_fits_with_flipped \
            = previous_tile.get_fits_with_transated(
                previous_flipped, previous_rotation)
        placement = Solution.place_by_tile(
            look_direction, previous_fits_with, False)
        if not placement:
            placement = Solution.place_by_tile(
                look_direction, previous_fits_with_flipped, True)

        self.placements[y][x] = placement

    def place(self, nw_id, nw_flipped, nw_rotation):
        for x, y in [(x, y)
                     for y in range(self.size)
                     for x in range(self.size)]:
            if x == 0 and y == 0:
                self.placements[y][x] = (nw_id, nw_flipped, nw_rotation)
                continue
            self.place_by_previous(x, y)
        return self.placements

In [9]:
class Mosaic(object):
    def __init__(self, solution):
        self.mosaic_size = solution.size
        self.tiles = solution.tiles
        self.placements = solution.placements

    def get_tiles_as_placed(self):
        output = []
        for placement_line in self.placements:
            output.append(
                [self.tiles[tile_id].get_as_placed(flipped, rotation)
                 for tile_id, flipped, rotation in placement_line]
            )
        return output

    def get_mosaic(self):
        output = []
        for tile_line in self.get_tiles_as_placed():
            output_tile_line = ['']*len(tile_line[0])
            for i in range(len(output_tile_line)):
                output_tile_line[i] = ' '.join(
                    [tile[i] for tile in tile_line]
                )
            output.append('\n'.join(output_tile_line))
        return '\n\n'.join(output)

    def get_trimmed_tiles(self):
        output = []
        for trimmed_tile_line in self.get_tiles_as_placed():
            output_tile_line = []
            for tile in trimmed_tile_line:
                output_tile = []
                for line in tile[1:-1]:
                    output_tile.append(line[1:-1])
                output_tile_line.append(output_tile)
            output.append(output_tile_line)
        return output

    def get_composite(self):
        output = []
        for trimmed_tile_line in self.get_trimmed_tiles():
            output_tile_line = ['']*len(trimmed_tile_line[0])
            for i in range(len(output_tile_line)):
                output_tile_line[i] = ''.join(
                    [trimmed_tile[i] for trimmed_tile in trimmed_tile_line]
                )
            output.append('\n'.join(output_tile_line))
        return '\n'.join(output)

    def find_monsters(self, flipped, rotation):
        composite_tile = Tile(['Tile 0000:']
                              + self.get_composite().splitlines())
        composite = composite_tile.get_as_placed(flipped, rotation)
        monster_idx = [[i for i, ch in enumerate(line) if ch == '#']
                       for line in monster_pattern]
        monsters = 0
        for i in range(len(composite) - len(monster_pattern) + 1):
            for j in range(len(composite) - len(monster_pattern[0]) + 1):
                no_monster = False
                for m_i in range(len(monster_idx)):
                    no_monster = any([True for idx in monster_idx[m_i]
                                      if composite[i + m_i][j + idx] != '#'])
                    if no_monster:
                        break
                if not no_monster:
                    monsters += 1      

        roughness = sum([line.count('#') for line in composite])
        roughness -= monsters*sum([len(idx_line) for idx_line in monster_idx])

        return monsters, roughness, flipped, rotation

    def find_any_monsters(self):
        results = []
        for flipped in {True, False}:
            for rotation in range(4):
                results.append(self.find_monsters(flipped, rotation))
        return results

66:34: W291 trailing whitespace


In [10]:
tiles = parse(testdata)
solution = Solution(tiles)
corner_ids = solution.fit_tiles()
print(f'corner tiles: {corner_ids}')
placements = solution.place(1951, True, 3)
print(placements)
# print(tiles[1951].get_as_placed(True, 3))

id: 2311, fits_with: [(WEST, 1951, EAST), (NORTH, 1427, SOUTH)], fits_with_flipped: [(EAST, 3079, WEST)]
id: 1951, fits_with: [(EAST, 2311, WEST), (NORTH, 2729, SOUTH)], fits_with_flipped: []
id: 1171, fits_with: [(EAST, 1489, EAST), (NORTH, 2473, WEST)], fits_with_flipped: []
id: 1427, fits_with: [(SOUTH, 2311, NORTH), (NORTH, 1489, SOUTH), (EAST, 2473, SOUTH), (WEST, 2729, EAST)], fits_with_flipped: []
id: 1489, fits_with: [(EAST, 1171, EAST), (SOUTH, 1427, NORTH), (WEST, 2971, EAST)], fits_with_flipped: []
id: 2473, fits_with: [(WEST, 1171, NORTH), (SOUTH, 1427, EAST)], fits_with_flipped: [(EAST, 3079, SOUTH)]
id: 2971, fits_with: [(EAST, 1489, WEST), (SOUTH, 2729, NORTH)], fits_with_flipped: []
id: 2729, fits_with: [(SOUTH, 1951, NORTH), (EAST, 1427, WEST), (NORTH, 2971, SOUTH)], fits_with_flipped: []
id: 3079, fits_with: [], fits_with_flipped: [(WEST, 2311, EAST), (SOUTH, 2473, EAST)]
corner tiles: [1951, 1171, 2971, 3079]
[[(1951, True, 3), (2311, True, 3), (3079, False, 0)], [(2

In [11]:
mosaic = Mosaic(solution)
print(mosaic.get_mosaic())
print()
composite_tile = Tile(['Tile 0000:'] + mosaic.get_composite().splitlines())
print('\n'.join(composite_tile.get_as_placed(True, 0)))
print(mosaic.find_any_monsters())

#...##.#.. ..###..### #.#.#####.
..#.#..#.# ###...#.#. .#..######
.###....#. ..#....#.. ..#.......
###.##.##. .#.#.#..## ######....
.###.##### ##...#.### ####.#..#.
.##.#....# ##.##.###. .#...#.##.
#...###### ####.#...# #.#####.##
.....#..## #...##..#. ..#.###...
#.####...# ##..#..... ..#.......
#.##...##. ..##.#..#. ..#.###...

#.##...##. ..##.#..#. ..#.###...
##..#.##.. ..#..###.# ##.##....#
##.####... .#.####.#. ..#.###..#
####.#.#.. ...#.##### ###.#..###
.#.####... ...##..##. .######.##
.##..##.#. ....#...## #.#.#.#...
....#..#.# #.#.#.##.# #.###.###.
..#.#..... .#.##.#..# #.###.##..
####.#.... .#..#.##.. .######...
...#.#.#.# ###.##.#.. .##...####

...#.#.#.# ###.##.#.. .##...####
..#.#.###. ..##.##.## #..#.##..#
..####.### ##.#...##. .#.#..#.##
#..#.#..#. ...#.#.#.. .####.###.
.#..####.# #..#.#.#.# ####.###..
.#####..## #####...#. .##....##.
##.##..#.. ..#...#... .####...#.
#.#.###... .##..##... .####.##.#
#...###... ..##...#.. ...#..####
..#.#....# ##.#.#.... ...##.....

.####..

In [12]:
tiles = parse(inputdata)
solution = Solution(tiles)
corner_ids = solution.fit_tiles(silent=True)
print(f'corner tiles: {corner_ids}')
placements = solution.place(3187, True, 0)
print(placements)
mosaic = Mosaic(solution)
print(mosaic.get_mosaic())
print()
composite_tile = Tile(['Tile 0000:'] + mosaic.get_composite().splitlines())
print('\n'.join(composite_tile.get_as_placed(True, 0)))
#print(mosaic.find_any_monsters())
print(mosaic.find_monsters(False, 2))

.###.#..# #.##.#.... .##.#.##.. .#..###..# ######.... .##.##.#.# ###.....#. .####.###. .####....#

.#...#..## #.##..###. ...#.#.... ..###.#..# #.##.#.... .##.#.##.. .#..###..# ######.... .##.##.#.# ###.....#. .####.###. .####....#
....#..#.# #.#.#.#..# #.......## #......... .....#.#.# #.#..#.##. ...##.#..# #..#...... .....###.. ....##...# ####....#. .#.......#
#......... .........# #......... ..#....#.# #..####... ..###.#.#. .##....#.# #.###....# #.#...#... .#.......# #.....#.#. ......#..#
......#..# ##.#.....# #.#......# #.......#. .....##..# ##..##.... .....#...# ##........ .......... ....#..#.# #........# #.........
##.....#.# #..#...... ..#..#...# #.......#. .......#.. ..#....##. ...#....## #...#..... .#.....#.# #........# ##....#..# ##........
.........# #...#....# #......#.# #........# #....#.... ......#... .......... ..###..... ..#.#..... .#......## #.#..###.. ...#..#..#
.#.....#.. .........# #...#..#.. .#........ .........# #........# #..#.#...# #......... ........#. .....#..##

In [13]:
for tile in tiles.values():
    print(f'{tile.id}: {len(tile.fits_with) + len(tile.fits_with_flipped)}')

1559: 3
3253: 4
1307: 4
2999: 4
1847: 4
2731: 4
1453: 4
1093: 4
1913: 4
3463: 4
3581: 4
3923: 4
3613: 3
3931: 4
3407: 4
1823: 4
1709: 4
1097: 4
2339: 3
2791: 4
1637: 4
2843: 3
1109: 3
3061: 4
3517: 4
3833: 4
1129: 4
1951: 4
2039: 4
3643: 3
2711: 4
2423: 3
1663: 4
1861: 3
3659: 4
1733: 4
3271: 3
3761: 4
3449: 3
2861: 3
1877: 3
1091: 3
1741: 4
2789: 4
2017: 4
2357: 4
3803: 3
1223: 4
1657: 4
3187: 2
2341: 3
3461: 4
2767: 3
3137: 3
3011: 4
3677: 4
3559: 4
1259: 4
3019: 4
2377: 4
1889: 4
2677: 4
3539: 4
1283: 4
1181: 4
1721: 4
3533: 3
1871: 4
2239: 2
3491: 3
3389: 4
1423: 3
2251: 3
2417: 4
2113: 4
2297: 3
3631: 3
1277: 3
1051: 4
2137: 4
2089: 4
3433: 3
2099: 4
2179: 3
3229: 4
1327: 4
2503: 2
2351: 3
1019: 3
2521: 3
1627: 4
2347: 3
3413: 4
2647: 4
1039: 4
2053: 4
2909: 4
1567: 4
2687: 4
3121: 4
3547: 4
1787: 4
1811: 4
2293: 3
1933: 4
2659: 3
2069: 4
2879: 4
2957: 4
2029: 3
1607: 3
3943: 4
3593: 4
1583: 4
3709: 4
1667: 4
3851: 2
3083: 4
2927: 4
2837: 3
1499: 3
2857: 4
1237: 4
3221: 4
2531: 4
